<a href="https://colab.research.google.com/github/Abhishek416/-Power-BI-Projects/blob/main/sle_track_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import plotly.express as px

In [ ]:
df = pd.read_csv('lup_data_19-04-24.csv')

In [ ]:
df['Disease state'].unique()

array(['Healthy', 'SLE'], dtype=object)

In [ ]:
df1 = df[df['Disease state'] == 'Healthy']
df2 = df[df['Disease state'] == 'SLE']

In [ ]:
df1.head()

,Subject,Visit,Visit_count,Cumulative_time,Days_since_diagnosis,Days_since_last_visit,Days_between_diagnosis_and_last_visit,Gender,Race,Age,...,Fever,Thrombocytopenia,Leukopenia,Renal,Musculoskeletal,Serology,Sledai_component_class,Sledaic_lmm2,Neph_treat_lmm3,Disease state
0,BAY-H377,1,1,Not Applicable,Not Applicable,Not Applicable,Not Applicable,F,AA,10.0,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Healthy
1,BAY-H290,1,1,Not Applicable,Not Applicable,Not Applicable,Not Applicable,F,H,13.0,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Healthy
2,BAY-H303,1,1,Not Applicable,Not Applicable,Not Applicable,Not Applicable,F,C,10.0,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Healthy
3,BAY-H380,1,1,Not Applicable,Not Applicable,Not Applicable,Not Applicable,F,AA,15.0,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Healthy
4,BAY-H306,1,1,Not Applicable,Not Applicable,Not Applicable,Not Applicable,F,H,6.0,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Healthy


In [ ]:
df1.replace('Not Applicable', np.nan, inplace=True)

<ipython-input-187-9907a3783da0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.replace('Not Applicable', np.nan, inplace=True)


In [ ]:
df2.replace(['Data Not Available', 'Unk.', 'None', '(-)'], np.nan, inplace=True)

<ipython-input-188-c6cdb562dca0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.replace(['Data Not Available', 'Unk.', 'None', '(-)'], np.nan, inplace=True)


In [ ]:
df1.shape

(72, 81)

In [ ]:
df2.shape

(924, 81)

In [ ]:
for col in df2.columns[1:-1]:
    try:
      df2[col] = pd.to_numeric(df2[col])
      df1[col] = pd.to_numeric(df1[col])
    except ValueError:
        print(f"Skipping column '{col}' due to ValueError")

Skipping column 'Gender' due to ValueError
Skipping column 'Race' due to ValueError
Skipping column 'Biopsy_history' due to ValueError
Skipping column 'Ds_dna' due to ValueError
Skipping column 'Treatment' due to ValueError
Skipping column 'Treatment_lmm1' due to ValueError
Skipping column 'Sledai_component_class' due to ValueError
Skipping column 'Sledaic_lmm2' due to ValueError
Skipping column 'Neph_treat_lmm3' due to ValueError


<ipython-input-191-b6af24176469>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[col] = pd.to_numeric(df2[col])
<ipython-input-191-b6af24176469>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[col] = pd.to_numeric(df1[col])


In [ ]:
df1.isnull().sum()

Subject                    0
Visit                      0
Visit_count                0
Cumulative_time           72
Days_since_diagnosis      72
                          ..
Serology                  72
Sledai_component_class    72
Sledaic_lmm2              72
Neph_treat_lmm3           72
Disease state              0
Length: 81, dtype: int64

In [ ]:
df2.isnull().sum()

Subject                     0
Visit                       1
Visit_count                 0
Cumulative_time             0
Days_since_diagnosis        3
                         ... 
Serology                    0
Sledai_component_class    104
Sledaic_lmm2              104
Neph_treat_lmm3           185
Disease state               0
Length: 81, dtype: int64

In [ ]:
# Fill missing values with mean
mean_values = round(df2.mean(numeric_only=True), 2)
df2.fillna(mean_values, inplace=True)

<ipython-input-194-fa1c231923a7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.fillna(mean_values, inplace=True)


In [ ]:
# Replace null values in string columns with most frequent value
string_cols = df2.select_dtypes(include=['object']).columns
for col in string_cols:
    most_common_value = df2[col].mode()[0]
    df2[col].fillna(most_common_value, inplace=True)

<ipython-input-195-80dd60e21938>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[col].fillna(most_common_value, inplace=True)


In [ ]:
df2.columns[df2.isnull().sum() >= 1]

Index([], dtype='object')

In [ ]:
def calculate_average(value):
    if isinstance(value, str) and ':' in value:
        parts = value.split(':')
        try:
            avg = (int(parts[0]) + int(parts[1])) / 2
            return avg
        except ValueError:
            return value

df2['Ds_dna'] = df2['Ds_dna'].apply(calculate_average)

<ipython-input-197-28b562448288>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Ds_dna'] = df2['Ds_dna'].apply(calculate_average)


In [ ]:
for col in df2.columns:
    if(df2[col].dtype != "object"):
        df1[col].fillna(df2[col].max()+1, inplace=True)
    else:
        df1[col].fillna("None", inplace=True)

<ipython-input-198-a634a761723f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[col].fillna("None", inplace=True)
<ipython-input-198-a634a761723f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[col].fillna(df2[col].max()+1, inplace=True)


In [ ]:
df = pd.concat([df1, df2], ignore_index=True)

In [ ]:
df.head()

,Subject,Visit,Visit_count,Cumulative_time,Days_since_diagnosis,Days_since_last_visit,Days_between_diagnosis_and_last_visit,Gender,Race,Age,...,Fever,Thrombocytopenia,Leukopenia,Renal,Musculoskeletal,Serology,Sledai_component_class,Sledaic_lmm2,Neph_treat_lmm3,Disease state
0,BAY-H377,1.0,1,1413.0,4506.0,736.0,4506.0,F,AA,10.0,...,2.0,2.0,2.0,2.0,2.0,2.0,None,None,None,Healthy
1,BAY-H290,1.0,1,1413.0,4506.0,736.0,4506.0,F,H,13.0,...,2.0,2.0,2.0,2.0,2.0,2.0,None,None,None,Healthy
2,BAY-H303,1.0,1,1413.0,4506.0,736.0,4506.0,F,C,10.0,...,2.0,2.0,2.0,2.0,2.0,2.0,None,None,None,Healthy
3,BAY-H380,1.0,1,1413.0,4506.0,736.0,4506.0,F,AA,15.0,...,2.0,2.0,2.0,2.0,2.0,2.0,None,None,None,Healthy
4,BAY-H306,1.0,1,1413.0,4506.0,736.0,4506.0,F,H,6.0,...,2.0,2.0,2.0,2.0,2.0,2.0,None,None,None,Healthy


In [ ]:
df.columns[df.isnull().sum() >= 1]

Index(['Ds_dna'], dtype='object')

In [ ]:
df['Ds_dna'].unique()

array([2.04815e+04,         nan, 5.50000e+00, 2.05000e+01, 1.05000e+01,
       2.04805e+04, 5.12050e+03])

In [ ]:
df['Ds_dna'].fillna(df['Ds_dna'].mean(), inplace=True)

In [ ]:
df.to_csv('output.csv', index=False)

In [ ]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [ ]:
X_label_encoders = []
for col in df2.select_dtypes(include=['object']).columns[:-1]:
    label_encoder = LabelEncoder()
    X[col] = label_encoder.fit_transform(X[col])
    X_label_encoders.append(label_encoder)

y_label_encoder = LabelEncoder()
y = y_label_encoder.fit_transform(y)

In [ ]:
# Standardize features
scaler_X = StandardScaler()
X_std = scaler_X.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score
# from xgboost import XGBClassifier

# xgb_clf = XGBClassifier()

# # Train the classifier
# xgb_clf.fit(X_train, y_train)

# # Make predictions
# y_pred = xgb_clf.predict(X_test)

# # Evaluate accuracy
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy}")

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt_classifier = DecisionTreeClassifier()

In [ ]:
dt_classifier.fit(X_train, y_train)

DecisionTreeClassifier()

In [ ]:
y_pred = dt_classifier.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 1.0
